In [1]:
import mlrun

In [59]:
project = mlrun.get_or_create_project(
    "heart-disease-classifier",
    parameters={
        "base_image" : "mlrun/mlrun",
        "requirements_file" : "requirements.txt",
        "force_build" : False,
        "source" : "v3io:///bigdata/testing1234.zip"
    }
)

Project Source: v3io:///bigdata/testing1234.zip
Exporting project as zip archive to v3io:///bigdata/testing1234.zip...
> 2025-01-07 23:46:52,702 [info] Project loaded successfully: {"project_name":"heart-disease-classifier"}


In [20]:
import pandas as pd

dataset = project.log_dataset(key="churn", df=pd.read_csv("data/churn_dataset_w_sentiment.csv"))
dataset.uri

'store://datasets/heart-disease-classifier/churn#0@14588ba6-9b04-4d00-937b-c4b6c11077f2'

In [38]:
train_run_id = project.run(
    name="main",
    arguments={
        'source_url': dataset.uri,
        'label_column': 'churn',
        'sentiment_column' : "sentiment_label",
        'ordinal_columns' : ["international_plan", "voice_mail_plan"],
        'drop_columns' : ["area_code", "chat_log", "state"],
        'allow_validation_failure': True,
        'model_name' : 'model'
    },
    dirty=True,
    watch=True
)

> 2025-01-07 23:38:44,029 [info] Pipeline submitted successfully: {"id":"e7ec98cb-32e5-473a-a4da-da9e2c3ffd38","pipeline_name":"heart-disease-classifier-main 2025-01-07 23-38-43"}
> 2025-01-07 23:38:44,030 [info] Pipeline run id=e7ec98cb-32e5-473a-a4da-da9e2c3ffd38, check UI for progress


> 2025-01-07 23:38:44,088 [info] Started run workflow heart-disease-classifier-main with run id = 'e7ec98cb-32e5-473a-a4da-da9e2c3ffd38' by kfp engine
> 2025-01-07 23:38:44,089 [info] Waiting for pipeline run completion: {"project":"heart-disease-classifier","run_id":"e7ec98cb-32e5-473a-a4da-da9e2c3ffd38"}


uid,start,state,kind,name,parameters,results
...118c0619,Jan 07 23:42:26,completed,run,validate-validate-model,model_path=store://models/heart-disease-classifier/model#1:latest@e7ec98cb-32e5-473a-a4da-da9e2c3ffd38label_column=churnallow_validation_failure=True,passed_suite=False
...acf2ba31,Jan 07 23:41:06,completed,run,train,label_column=churnmodel_name=model,best_iteration=4accuracy=0.9583333333333334f1_score=0.8705882352941177precision_score=0.9024390243902439recall_score=0.8409090909090909model_uri=store://models/heart-disease-classifier/model#1:latest@e7ec98cb-32e5-473a-a4da-da9e2c3ffd38
...68c5dabb,Jan 07 23:40:35,completed,run,validate-validate-train-test-split,label_column=churnallow_validation_failure=True,passed_suite=False
...a951a57c,Jan 07 23:40:06,completed,run,data-process-data,"label_column=churntest_size=0.1ordinal_columns=['international_plan', 'voice_mail_plan']drop_columns=['area_code', 'chat_log', 'state']sentiment_column=sentiment_label",
...be0f2d11,Jan 07 23:39:16,completed,run,validate-validate-data-integrity,label_column=churnallow_validation_failure=True,passed_suite=False
...d5cefca9,Jan 07 23:38:50,completed,run,data-get-data,,


In [39]:
serving_fn = project.get_function("serving")

In [57]:
test = mlrun.get_dataitem("store://datasets/heart-disease-classifier/data-process-data_test#0:latest").as_df()
test = test.drop("churn", axis=1)
data = test.head(5).to_dict(orient="split")["data"]

In [61]:
%%time
serving_fn.invoke(
    "/v2/models/model/predict",
    body={
        "inputs": data
    }
)

> 2025-01-08 17:07:23,574 [info] Invoking function: {"method":"POST","path":"http://nuclio-heart-disease-classifier-serving.default-tenant.svc.cluster.local:8080/v2/models/model/predict"}
CPU times: user 10.7 ms, sys: 0 ns, total: 10.7 ms
Wall time: 50.8 ms


{'id': '4ab6364f-c221-439b-8a46-553bc15fcb42',
 'model_name': 'model',
 'outputs': [0, 0, 0, 0, 0],
 'timestamp': '2025-01-08 17:07:23.584059+00:00',
 'model_version': 'latest'}